In [1]:
# train_fmnist_keras.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Hyperparams
BATCH_SIZE = 128
EPOCHS = 30
LR = 1e-3
MODEL_PATH = "fmnist_keras_model.h5"

# Load Fashion MNIST from Keras datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize and reshape
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
# add channel axis
x_train = np.expand_dims(x_train, -1)  # (N,28,28,1)
x_test  = np.expand_dims(x_test, -1)

# One-hot labels (optional, using sparse_categorical_crossentropy we could keep ints)
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat  = to_categorical(y_test, num_classes)

# Build model (small CNN)
def build_model():
    inp = layers.Input(shape=(28,28,1))
    x = layers.Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)   # 14x14
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)   # 7x7
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs=inp, outputs=out)
    return model

model = build_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor="val_loss")
]

# Train
history = model.fit(
    x_train, y_train_cat,
    validation_split=0.1,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=2
)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=2)
print(f"Keras test loss: {test_loss:.4f}, test accuracy: {test_acc:.4f}")

# Save (already saved by ModelCheckpoint)
print(f"Model saved to {os.path.abspath(MODEL_PATH)}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 897,290 (3.42 MB)

 Trainable params: 897,098 (3.42 MB)

...

Epoch 1/30


422/422 - 15s - 36ms/step - accuracy: 0.8404 - loss: 0.4590 - val_accuracy: 0.6820 - val_loss: 1.1796
Epoch 2/30


422/422 - 2s - 6ms/step - accuracy: 0.8995 - loss: 0.2832 - val_accuracy: 0.9083 - val_loss: 0.2484
Epoch 3/30
422/422 - 2s - 6ms/step - accuracy: 0.9145 - loss: 0.2348 - val_accuracy: 0.8910 - val_loss: 0.3069
Epoch 4/30


422/422 - 3s - 6ms/step - accuracy: 0.9273 - loss: 0.1999 - val_accuracy: 0.9148 - val_loss: 0.2302
Epoch 5/30
422/422 - 2s - 6ms/step - accuracy: 0.9331 - loss: 0.1799 - val_accuracy: 0.9087 - val_loss: 0.2403
Epoch 6/30


422/422 - 3s - 7ms/step - accuracy: 0.9407 - loss: 0.1594 - val_accuracy: 0.9208 - val_loss: 0.2242
Epoch 7/30
422/422 - 2s - 6ms/step - accuracy: 0.9462 - loss: 0.1437 - val_accuracy: 0.9182 - val_loss: 0.2441
Epoch 8/30
422/422 - 2s - 6ms/step - accuracy: 0.9513 - loss: 0.1273 - val_accuracy: 0.9223 - val_loss: 0.2419
Epoch 9/30
422/422 - 2s - 6ms/step - accuracy: 0.9575 - loss: 0.1101 - val_accuracy: 0.9182 - val_loss: 0.2603
Epoch 10/30
422/422 - 3s - 6ms/step - accuracy: 0.9612 - loss: 0.1031 - val_accuracy: 0.9033 - val_loss: 0.3309
Epoch 11/30
422/422 - 5s - 12ms/step - accuracy: 0.9630 - loss: 0.0954 - val_accuracy: 0.9183 - val_loss: 0.2603
Epoch 12/30
422/422 - 2s - 6ms/step - accuracy: 0.9642 - loss: 0.0920 - val_accuracy: 0.9168 - val_loss: 0.2933
313/313 - 2s - 6ms/step - accuracy: 0.9157 - loss: 0.2592
Keras test loss: 0.2592, test accuracy: 0.9157
Model saved to /content/fmnist_keras_model.h5
